<a href="https://colab.research.google.com/github/thmswhelan/UCDPA_ThomasWhelan/blob/main/PropertyPricesIreland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import pandas as pd

url = "https://raw.githubusercontent.com/thmswhelan/UCDPA_ThomasWhelan/main/Property_Price_Register_Ireland-28-05-2021.csv"

df_house = pd.read_csv(url)

In [152]:
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [153]:
df_house.head()

,SALE_DATE,ADDRESS,POSTAL_CODE,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,PROPERTY_SIZE_DESC
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,343000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
1,2010-01-03,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,0,1,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,438500.0,0,0,Second-Hand Dwelling house /Apartment,NaN
3,2010-01-04,"1 The Haven, Mornington",NaN,Meath,400000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
4,2010-01-04,"11 Melville Heights, Kilkenny",NaN,Kilkenny,160000.0,0,0,Second-Hand Dwelling house /Apartment,NaN


In [154]:
pip install requests beautifulsoup4

In [155]:
import requests
import pandas as pd

url = "https://www.ecb.europa.eu/stats/policy_and_exchange_rates/key_ecb_interest_rates/html/index.en.html"
response = requests.get(url)

if response.status_code == 200:
    page_content = response.content
    dfs = pd.read_html(page_content)  # Returns a list of DataFrames from HTML tables
    table_df = dfs[0]  # Assuming the first table is the one with the data
    # You might need to adjust the index if the target table is at a different position.
    print(table_df)
else:
    print("Failed to fetch the web page.")

                              Date (with effect from)  \
                                   Unnamed: 0_level_1   
0                                                2023   
1                                                2023   
2                                                2023   
3                                                2023   
4                                                2023   
5                                                2022   
6                                                2022   
7                                                2022   
8                                                2022   
9                                                2019   
10                                               2016   
11                                               2015   
12                                               2014   
13                                                NaN   
14                                               2013   
15                             

In [156]:

# Create a DataFrame from the provided data
df = pd.DataFrame(table_df)
df.head()


Date (with effect from)                      Deposit facility  \
       Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
0                    2023             2 Aug.               3.75   
1                    2023            21 Jun.               3.50   
2                    2023             10 May               3.25   
3                    2023            22 Mar.               3.00   
4                    2023             8 Feb.               2.50   

    Main refinancing operations                                         \
  Fixed rate tenders Fixed rate Variable rate tenders Minimum bid rate   
0                          4.25                                      -   
1                          4.00                                      -   
2                          3.75                                      -   
3                          3.50                                      -   
4                          3.00                                      -   

  Marginal lending facility  
         Unnamed: 5_level_1  
0                      4.50  
1                      4.25  
2                      4.00  
3                      3.75  
4                      3.25

In [157]:
columns_to_drop = ["Deposit facility", "Main refinancing operations"]
df_dropped = df.drop(columns=columns_to_drop)
df_dropped.head()

Date (with effect from)                    Marginal lending facility
       Unnamed: 0_level_1 Unnamed: 1_level_1        Unnamed: 5_level_1
0                    2023             2 Aug.                      4.50
1                    2023            21 Jun.                      4.25
2                    2023             10 May                      4.00
3                    2023            22 Mar.                      3.75
4                    2023             8 Feb.                      3.25

In [158]:
df_keep = pd.DataFrame(df_dropped)

# Flatten the multi-level column headers
df_keep.columns = df_keep.columns.map('_'.join)

# Select the specific columns you want
desired_columns = ["Date (with effect from)_Unnamed: 0_level_1", "Date (with effect from)_Unnamed: 1_level_1", "Marginal lending facility_Unnamed: 5_level_1"]
filtered_df = df_keep[desired_columns]


In [173]:
df_interest= pd.DataFrame(filtered_df)
print(df_interest)

           Date (with effect from)_Unnamed: 0_level_1  \
0                                                2023   
1                                                2023   
2                                                2023   
3                                                2023   
4                                                2023   
5                                                2022   
6                                                2022   
7                                                2022   
8                                                2022   
9                                                2019   
10                                               2016   
11                                               2015   
12                                               2014   
13                                                NaN   
14                                               2013   
15                                                NaN   
16                             

In [179]:
column_name = 'Date (with effect from)_Unnamed: 1_level_1'
pattern_to_remove = r'\.\d$|\.$|\.\s\d$'  # Regular expression to match ".#" at the end of a string

df_interest[column_name] = df_interest[column_name].str.replace(pattern_to_remove, '')
print(df_interest)

           Date (with effect from)_Unnamed: 0_level_1  \
0                                                2023   
1                                                2023   
2                                                2023   
3                                                2023   
4                                                2023   
5                                                2022   
6                                                2022   
7                                                2022   
8                                                2022   
9                                                2019   
10                                               2016   
11                                               2015   
12                                               2014   
13                                                NaN   
14                                               2013   
15                                                NaN   
16                             

<ipython-input-179-5f6760d074c3>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_interest[column_name] = df_interest[column_name].str.replace(pattern_to_remove, '')


In [180]:
df_interest = df_interest.fillna(method='ffill')

In [182]:
df_interest['Combined']=df_interest['Date (with effect from)_Unnamed: 1_level_1'] + " "+ df_interest['Date (with effect from)_Unnamed: 0_level_1']
df_interest.head()

,Date (with effect from)_Unnamed: 0_level_1,Date (with effect from)_Unnamed: 1_level_1,Marginal lending facility_Unnamed: 5_level_1,Combined
0,2023,2 Aug,4.50,2 Aug 2023
1,2023,21 Jun,4.25,21 Jun 2023
2,2023,10 May,4.00,10 May 2023
3,2023,22 Mar,3.75,22 Mar 2023
4,2023,8 Feb,3.25,8 Feb 2023


In [183]:
columns_to_drop = ['Date (with effect from)_Unnamed: 0_level_1', 'Date (with effect from)_Unnamed: 1_level_1']
df_dropped = df_interest.drop(columns=columns_to_drop)

In [184]:
df_dropped.head()

,Marginal lending facility_Unnamed: 5_level_1,Combined
0,4.50,2 Aug 2023
1,4.25,21 Jun 2023
2,4.00,10 May 2023
3,3.75,22 Mar 2023
4,3.25,8 Feb 2023


In [185]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Marginal lending facility_Unnamed: 5_level_1  60 non-null     object
 1   Combined                                      60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


In [186]:
df_dropped.dtypes

Marginal lending facility_Unnamed: 5_level_1    object
Combined                                        object
dtype: object

In [187]:
print(df_dropped)

         Marginal lending facility_Unnamed: 5_level_1  \
0                                                4.50   
1                                                4.25   
2                                                4.00   
3                                                3.75   
4                                                3.25   
5                                                2.75   
6                                                2.25   
7                                                1.50   
8                                                0.75   
9                                                0.25   
10                                               0.25   
11                                               0.30   
12                                               0.30   
13                                               0.40   
14                                               0.75   
15                                               1.00   
16                             

In [188]:
df_dropped = df_dropped.drop(index=df_dropped.index[-2:])
print(df_dropped)

   Marginal lending facility_Unnamed: 5_level_1     Combined
0                                          4.50   2 Aug 2023
1                                          4.25  21 Jun 2023
2                                          4.00  10 May 2023
3                                          3.75  22 Mar 2023
4                                          3.25   8 Feb 2023
5                                          2.75  21 Dec 2022
6                                          2.25   2 Nov 2022
7                                          1.50  14 Sep 2022
8                                          0.75  27 Jul 2022
9                                          0.25  18 Sep 2019
10                                         0.25  16 Mar 2016
11                                         0.30   9 Dec 2015
12                                         0.30  10 Sep 2014
13                                         0.40  11 Jun 2014
14                                         0.75  13 Nov 2013
15                      

In [189]:
df_dropped['Marginal lending facility_Unnamed: 5_level_1'] = df_dropped['Marginal lending facility_Unnamed: 5_level_1'].astype(float)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 2 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Marginal lending facility_Unnamed: 5_level_1  58 non-null     float64
 1   Combined                                      58 non-null     object 
dtypes: float64(1), object(1)
memory usage: 1.0+ KB
